# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,38.14,97,99,6.49,AR,1704701173
1,1,whitehorse,60.7161,-135.0538,6.03,82,100,21.85,CA,1704701237
2,2,longyearbyen,78.2186,15.6401,6.64,72,75,12.66,SJ,1704701246
3,3,waitangi,-43.9535,-176.5597,55.81,82,17,1.01,NZ,1704701397
4,4,yellowknife,62.4560,-114.3525,-30.71,99,96,4.72,CA,1704701398


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_condition = city_data_df.loc[
    (city_data_df["Max Temp"] >=35.5) &
    (city_data_df["Max Temp"] >=25.5) & 
    (city_data_df["Wind Speed"]<=20) & 
    (city_data_df["Cloudiness"]<=15)]

# Drop any rows with null values
ideal_condition_drop = ideal_condition.dropna()

# Display sample data
display(ideal_condition)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,lompoc,34.6391,-120.4579,44.69,73,0,4.61,US,1704701404
18,18,nakamura,32.9833,132.9333,43.00,59,0,0.83,JP,1704701207
27,27,howrah,22.5892,88.3103,75.20,64,1,6.91,IN,1704701317
30,30,salalah,17.0151,54.0924,80.69,32,0,18.41,OM,1704701426
38,38,al jawf,29.5000,38.7500,68.13,39,1,8.52,SA,1704701433
...,...,...,...,...,...,...,...,...,...,...
569,569,rio gallegos,-51.6226,-69.2181,48.25,46,0,11.50,AR,1704701899
572,572,greenville,35.6127,-77.3663,39.22,90,0,0.00,US,1704701901
576,576,inhambane,-23.8650,35.3833,85.53,52,5,5.26,MZ,1704701904
577,577,galliano,29.4422,-90.2992,55.40,76,0,5.75,US,1704701904


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,97,
1,whitehorse,CA,60.7161,-135.0538,82,
2,longyearbyen,SJ,78.2186,15.6401,72,
3,waitangi,NZ,-43.9535,-176.5597,82,
4,yellowknife,CA,62.4560,-114.3525,99,
...,...,...,...,...,...,...
576,inhambane,MZ,-23.8650,35.3833,52,
577,galliano,US,29.4422,-90.2992,76,
578,bonga,ET,7.2833,36.2333,74,
579,coronel moldes,AR,-33.6227,-64.5971,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",   
    "apiKey": geoapify_key,
    "limit": 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
tofol - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
paracuru - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
oamaru - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
santa maria do suacui - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
kamaishi - nearest hotel: No hotel found
nakamura - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
otjimbingwe - nearest hotel: No hotel found
alice springs - n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,97,No hotel found
1,whitehorse,CA,60.7161,-135.0538,82,No hotel found
2,longyearbyen,SJ,78.2186,15.6401,72,No hotel found
3,waitangi,NZ,-43.9535,-176.5597,82,No hotel found
4,yellowknife,CA,62.4560,-114.3525,99,No hotel found
...,...,...,...,...,...,...
576,inhambane,MZ,-23.8650,35.3833,52,No hotel found
577,galliano,US,29.4422,-90.2992,76,No hotel found
578,bonga,ET,7.2833,36.2333,74,No hotel found
579,coronel moldes,AR,-33.6227,-64.5971,98,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

city =['salalah', 'behbahan']

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    alpha = 0.5,
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"],
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)